In [1]:
#!pip install lightgbm
#!pip install scikit-learn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [12]:
import pandas as pd
df = pd.read_csv("hospital_readmissions.csv")

print("Dataset Shape:", df.shape)
df.head()



Dataset Shape: (25000, 17)


,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18,2,0,0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,no
1,[70-80),3,34,2,13,0,0,0,Other,Other,Other,Other,no,no,no,yes,no
2,[50-60),5,45,0,18,0,0,0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,yes
3,[70-80),2,36,0,12,1,0,0,Missing,Circulatory,Other,Diabetes,no,no,yes,yes,yes
4,[60-70),1,42,0,7,0,0,0,InternalMedicine,Other,Circulatory,Respiratory,no,no,no,yes,no


In [16]:
print(df["readmitted"].unique())

[nan]


In [19]:
df["readmitted"] = df["readmitted"].map({"Yes": 1, "No": 0})
df["readmitted"] = df["readmitted"].fillna(0)

In [20]:
X = df.drop("readmitted", axis=1)
y = df["readmitted"].astype(int)

In [21]:

cat_cols = X.select_dtypes(include=['object']).columns
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# ✅ Fix: Clean column names to remove special JSON characters
X.columns = X.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

print("Final Feature Shape:", X.shape)

Final Feature Shape: (25000, 45)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [23]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


In [24]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_class = (y_pred >= 0.5).astype(int)

print("\n✅ Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_class))
print("AUC Score:", roc_auc_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred_class))


✅ Model Evaluation:
Accuracy: 1.0
AUC Score: nan

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5000

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000



C:\Users\keert\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
